In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from functools import partial
import geopy
import time
from geopy.extra.rate_limiter import RateLimiter
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [2]:
URL = "https://travelness.com/most-visited-cities-in-the-world"

In [3]:
# Header for request
HEADER = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en; q=0.5'})
geolocator = Nominatim(user_agent="geocode_locations_from_webpage")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5, max_retries=3)
geo_key = 'e0e73007ebe240c6836e278bbc510da2'
geocoder = OpenCageGeocode(geo_key)

In [4]:
webpage = requests.get(URL, headers=HEADER)
webpage

<Response [200]>

In [5]:
soup = BeautifulSoup(webpage.content, "html.parser")

In [6]:
data = soup.find_all("td", attrs = {'class': 'has-text-align-left'})
len(data)

60

In [7]:
print(data[0].text, data[1].text, data[2].text)
print(data[-3].text, data[-2].text, data[-1].text)
# Use i % 3 == {0,1,2} in if-else for different data

1 Bangkok 22.78 Million Visitors
20 Hong Kong SAR 8.23 Million Visitors


In [8]:
location_description2 = geocoder.geocode(data[4].text)
len(location_description2)

8

In [9]:
location = geolocator.geocode(data[7].text, language="en")
print(location)
print(data[7].text)
latitude = str(location.raw['lat'])
longitude = str(location.raw['lon'])
lat_long = latitude + ","+ longitude 
location2 = geolocator.reverse(lat_long, language="en")
city = data[7].text
try:
    postcode = location2.raw['address']['postcode'].replace(" ","")
except:
    postcode = "Null"
country = location2.raw['address']['country'].upper()
country_code = location2.raw['address']['country_code']
print(city + "  " + country + "  "+ str(round(float(latitude),2)) + "  "+ str(round(float(longitude),2)) + "  " + country_code.upper() + "  " + postcode)
#location2.raw

London, Greater London, England, United Kingdom
London
London  UNITED KINGDOM  51.51  -0.13  GB  SW1A2DX


In [10]:
city_address = []
for i in range(60):
    if i % 3 == 1:

        # GETTING RAW DATA
        try:
            location_description = geocoder.geocode(data[i].text)
        except Exception:
            print(str(i) + " " + "No data found for location: " + str(data[i].text))
            continue
        
        # GETTING ADDRESS COMPONENTS
        city = data[i].text
        country = location_description[0]['components']['country']
        country_code = location_description[0]['components']['country_code']
        continent = location_description[0]['components']['continent']
        latitude = str(location_description[0]['geometry']['lat'])
        longitude = str(location_description[0]['geometry']['lng'])
        visitors = data[i+1].text.split(" ")[0]

        #GETTING POSTCODE
        lat_long = str(latitude) + ","+ str(longitude) 
        location2 = geolocator.reverse(lat_long, language="en")
        try:
            postcode = location2.raw['address']['postcode'].replace(" ","")
        except:
            postcode = "NULL"
        
        # ADDING VARIABLES TO DICTIONARY
        #print(str(i) +"  "+ city +"  " + country + "  " + country_code.upper() + "  " + continent + "  " + latitude + "  "+ longitude+ "  " + postcode + "  " + visitors)
        address_dict = {'City': city, 'Country': country, 'Country Code':country_code.upper(), 'Continent': continent, 'Latitude': str(round(float(latitude),2)), 'Longitude': str(round(float(longitude),2)), 'Postcode': postcode, 'Visitors (in Millions)': visitors}
        city_address.append(address_dict)
    
print(city_address)

[{'City': 'Bangkok', 'Country': 'Thailand', 'Country Code': 'TH', 'Continent': 'Asia', 'Latitude': '13.75', 'Longitude': '100.49', 'Postcode': '10200', 'Visitors (in Millions)': '22.78'}, {'City': 'Paris', 'Country': 'France', 'Country Code': 'FR', 'Continent': 'Europe', 'Latitude': '48.86', 'Longitude': '2.32', 'Postcode': '75007', 'Visitors (in Millions)': '19.10'}, {'City': 'London', 'Country': 'United Kingdom', 'Country Code': 'GB', 'Continent': 'Europe', 'Latitude': '51.51', 'Longitude': '-0.13', 'Postcode': 'SW1A2DX', 'Visitors (in Millions)': '19.09'}, {'City': 'Dubai', 'Country': 'United Arab Emirates', 'Country Code': 'AE', 'Continent': 'Asia', 'Latitude': '25.07', 'Longitude': '55.19', 'Postcode': 'NULL', 'Visitors (in Millions)': '15.93'}, {'City': 'Singapore', 'Country': 'Singapore', 'Country Code': 'SG', 'Continent': 'Asia', 'Latitude': '1.36', 'Longitude': '103.82', 'Postcode': '578774', 'Visitors (in Millions)': '14.67'}, {'City': 'Kuala Lumpur', 'Country': 'Malaysia', '

In [11]:
#city_address = []
#for i in range(60):
#    if i % 3 == 1:
#
#        # GETTING LATITUDE & LONGITUDE
#        location = geolocator.geocode(data[i].text, language="en")
#        latitude = str(location.raw['lat'])
#        longitude = str(location.raw['lon'])
#        lat_long = latitude + ","+ longitude 
#        location2 = geolocator.reverse(lat_long, language="en")
#        
#        # ASSIGNING VARIABLES
#        city = data[i].text
#        country = location2.raw['address']['country']
#        country_code = location2.raw['address']['country_code']
#        try:
#            postcode = location2.raw['address']['postcode'].replace(" ","-")
#        except:
#            postcode = "NULL"
#        visitors = data[i+1].text.split(" ")[0]
#        
#        # ADDING VARIABLES TO DICTIONARY
#        # print(str(i) +"  "+ city +"  " + country + "  " + country_code.upper() + "  " + latitude + "  "+ longitude+ "  " + postcode + "  " + visitors)
#        address_dict = {'City': city, 'Country': country, 'Country Code':country_code.upper(), 'Latitude': str(round(float(latitude),2)), 'Longitude': str(round(float(longitude),2)), 'Postcode': postcode, 'Visitors (in Millions)': visitors}
#        city_address.append(address_dict)
#    
#print(city_address)

In [12]:
#ca_most_tourist_df_2 = pd.DataFrame.from_dict(city_address_2)
#ca_most_tourist_df_2

In [13]:
ca_most_tourist_df = pd.DataFrame.from_dict(city_address)
ca_most_tourist_df

,City,Country,Country Code,Continent,Latitude,Longitude,Postcode,Visitors (in Millions)
0,Bangkok,Thailand,TH,Asia,13.75,100.49,10200,22.78
1,Paris,France,FR,Europe,48.86,2.32,75007,19.10
2,London,United Kingdom,GB,Europe,51.51,-0.13,SW1A2DX,19.09
3,Dubai,United Arab Emirates,AE,Asia,25.07,55.19,NULL,15.93
4,Singapore,Singapore,SG,Asia,1.36,103.82,578774,14.67
5,Kuala Lumpur,Malaysia,MY,Asia,3.15,101.69,50730,13.79
6,New York,United States,US,North America,40.71,-74.01,10000,13.60
7,Istanbul,Turkey,TR,Europe,41.01,28.97,34126,13.40
8,Tokyo,Japan,JP,Asia,35.68,139.77,103-8230,12.93
9,Antalya,Turkey,TR,Asia,36.89,30.71,07100,12.41
